In [238]:
import pandas as pd
import networkx as nx
from collections import defaultdict
import heapq

In [181]:
df = pd.read_csv(r"AirportsClean.csv")

In [239]:
#Non ci è necessario tutto il dataset di dati, creiamo quindi un dataframe che sara composto solo dalle colonne Origin_airport,Destination_airport,Distance e Count.
##Ci rendiamo conto che alle volte per la stessa coppia Origin_airport,Destination_airport sono presenti distanze differenti (probabilmente per errori di rilevazione o a causa di modifiche dei territori nel tempo), 
##per risolvere sostituiamo il valore della distanza con il valore della distanza piu frequente (la moda) per coppia di Origin_airport,Destination_airport.
##Il campo count invece rappresentera il conteggio per ogni coppia.


# Calcolo della moda considerando le coppie come uguali indipendentemente dall'ordine
df['Airport_pair'] = df.apply(
    lambda row: tuple(sorted([row['Origin_airport'], row['Destination_airport']])), axis=1
)

# Calcola la moda della distanza per ogni coppia ordinata
pair_mode_distances = (
    df.groupby('Airport_pair')['Distance']
    .transform(lambda x: x.mode()[0])  # Prendi il valore più frequente (moda)
)

# Sostituisci la distanza con la moda calcolata
df['Distance_mode'] = pair_mode_distances

# Calcolo del conteggio per ciascuna coppia distinta di origine e destinazione
pair_counts = df.groupby(['Origin_airport', 'Destination_airport']).size().reset_index(name='Count')

# Crea un nuovo DataFrame con le coppie distinte, le distanze aggiornate e il conteggio
df_util = df[['Origin_airport', 'Destination_airport', 'Distance_mode']].drop_duplicates().rename(columns={'Distance_mode': 'Distance'})

# Aggiungo il conteggio al DataFrame distinct_airports
df_util = df_util.merge(pair_counts, on=['Origin_airport', 'Destination_airport'], how='left')

# Rimuovo la colonna temporanea Airport_pair
df_util = df_util.drop(columns=['Airport_pair'], errors='ignore')
df_util.to_csv("df_util.csv")
df_util.head()

# Funzione per costruire un grafo da un DataFrame
def build_graph(df):
    G = nx.DiGraph()
    for _, row in df.iterrows():
        G.add_edge(row['Origin_airport'], row['Destination_airport'], weight=row['Distance'])
    return G

# Costruzione del grafo orientato con pesi
build_graph(df_util)

In [97]:
# Concatenare le colonne Origin_airport e Destination_airport e rimuovere duplicati
airports = pd.concat([df['Origin_airport'], df['Destination_airport']]).drop_duplicates()

In [220]:
#Q1 degree nodi

# Numero totale di nodi (consideriamo origine e destinazione come un unico insieme di nodi)
n = len(airports)  # Numero totale di nodi

# Calcolo del grado uscente (out-degree)
out_degree = df_util.groupby('Origin_airport').size().reset_index(name='Out_degree')

# Calcolo del grado entrante (in-degree)
in_degree = df_util.groupby('Destination_airport').size().reset_index(name='In_degree')

# Unione dei risultati in un unico DataFrame
degree_df = pd.merge(out_degree, in_degree, 
                     left_on='Origin_airport', 
                     right_on='Destination_airport', 
                     how='outer', 
                     suffixes=('_origin', '_destination'))

# Riempire i valori NaN con 0
degree_df['Out_degree'] = degree_df['Out_degree'].fillna(0).astype(int)
degree_df['In_degree'] = degree_df['In_degree'].fillna(0).astype(int)

# Calcolo del grado totale per ogni nodo
degree_df['Total_degree'] = degree_df['Out_degree'] + degree_df['In_degree']

# Calcolo del grado medio normalizzato
degree_df['Normalized_degree'] = degree_df['Total_degree'] / (2 * (n - 1))

# Visualizzazione del risultato
print(degree_df[['Origin_airport', 'Out_degree', 'In_degree','Total_degree', 'Normalized_degree']].sort_values(by="Origin_airport"))


    Origin_airport  Out_degree  In_degree  Total_degree  Normalized_degree
0              1B1           1          1             2           0.001377
1              ABE         121        115           236           0.162534
2              ABI          62        126           188           0.129477
3              ABQ         196        181           377           0.259642
4              ABR          26         19            45           0.030992
..             ...         ...        ...           ...                ...
694            NaN           0          1             1           0.000689
696            NaN           0          1             1           0.000689
716            NaN           0          2             2           0.001377
718            NaN           0          1             1           0.000689
725            NaN           0          1             1           0.000689

[727 rows x 5 columns]


In [240]:
def calculate_degree_centrality(graph):
    """
    Calcola i gradi entranti, uscenti e normalizzati per ciascun nodo in un grafo.

    Parameters:
        graph (networkx.DiGraph): Il grafo rappresentante la rete di voli.

    Returns:
        pd.DataFrame: DataFrame con gradi entranti, uscenti e normalizzati per ogni nodo.
    """
    nodes = list(graph.nodes)
    n = len(nodes)

    # Calcolo del grado uscente (out-degree)
    out_degree = {node: len(list(graph.successors(node))) for node in nodes}

    # Calcolo del grado entrante (in-degree)
    in_degree = {node: len(list(graph.predecessors(node))) for node in nodes}

    # Creazione del DataFrame
    degree_data = []
    for node in nodes:
        out_d = out_degree.get(node, 0)
        in_d = in_degree.get(node, 0)
        total_d = out_d + in_d
        normalized_d = total_d / (2 * (n - 1)) if n > 1 else 0
        degree_data.append({
            'Airport': node,
            'Out_degree': out_d,
            'In_degree': in_d,
            'Total_degree': total_d,
            'Normalized_degree': normalized_d
        })

    return pd.DataFrame(degree_data)

degree_df = calculate_degree_centrality(G)
print(degree_df[['Airport', 'Out_degree', 'In_degree','Total_degree', 'Normalized_degree']].sort_values(by="Airport"))

    Airport  Out_degree  In_degree  Total_degree  Normalized_degree
513     1B1           1          1             2           0.001377
197     ABE         121        115           236           0.162534
145     ABI          62        126           188           0.129477
43      ABQ         196        181           377           0.259642
74      ABR          26         19            45           0.030992
..      ...         ...        ...           ...                ...
609     YKN           2          1             3           0.002066
204     YNG          92        105           197           0.135675
123     YUM          68         67           135           0.092975
683     ZXX           0          1             1           0.000689
565     ZZV           4          5             9           0.006198

[727 rows x 5 columns]


In [241]:
# Funzione per calcolare la Betweenness Centrality ponderata
def calculate_betweenness_centrality_weighted(graph):
    """
    Calcola la Betweenness Centrality ponderata per un grafo orientato.

    Parameters:
        graph (networkx.DiGraph): Il grafo rappresentante la rete di voli.

    Returns:
        dict: Dizionario contenente la Betweenness Centrality per ogni nodo.
    """
    centrality = defaultdict(float)
    nodes = list(graph.nodes)

    for s in nodes:  # Per ogni nodo sorgente
        # Inizializza strutture per percorsi più brevi
        sigma = defaultdict(int)  # Numero di percorsi più brevi
        sigma[s] = 1
        dist = defaultdict(lambda: float('inf'))  # Distanza inizializzata a infinito
        dist[s] = 0
        pred = defaultdict(list)  # Predecessori
        queue = []  # Priority queue per Dijkstra
        heapq.heappush(queue, (0, s))  # (distanza, nodo)
        stack = []

        # Calcolo dei percorsi più brevi (Dijkstra per grafi ponderati)
        while queue:
            d, v = heapq.heappop(queue)
            if dist[v] < d:
                continue
            stack.append(v)
            for w in graph.successors(v):
                weight = graph[v][w]['weight']
                if dist[w] > dist[v] + weight:  # Nuovo percorso più breve trovato
                    dist[w] = dist[v] + weight
                    heapq.heappush(queue, (dist[w], w))
                    sigma[w] = sigma[v]
                    pred[w] = [v]
                elif dist[w] == dist[v] + weight:  # Percorso altrettanto breve trovato
                    sigma[w] += sigma[v]
                    pred[w].append(v)

        # Accumula dipendenze
        dependency = defaultdict(float)
        while stack:
            w = stack.pop()
            for v in pred[w]:
                dependency[v] += (sigma[v] / sigma[w]) * (1 + dependency[w])
            if w != s:
                centrality[w] += dependency[w]

    # Normalizzazione (per grafi orientati)
    normalization_factor = (len(nodes) - 1) * (len(nodes) - 2)
    if normalization_factor > 0:
        for node in centrality:
            centrality[node] /= normalization_factor

    return dict(centrality)
#
## Calcolo della Betweenness Centrality ponderata
betweenness_centrality_weighted = calculate_betweenness_centrality_weighted(G)
#
## Mostra i risultati
betweenness_df_weighted = pd.DataFrame(
    betweenness_centrality_weighted.items(),
    columns=['Airport', 'Betweenness_Centrality']
)
print(betweenness_df_weighted.sort_values(by='Betweenness_Centrality', ascending=False, inplace=False))

    Airport  Betweenness_Centrality
456     YIP                0.323335
191     WFB                0.307999
190     KTN                0.219055
192     MTN                0.092484
410     ATL                0.079322
..      ...                     ...
0       AWX                0.000000
14      HUA                0.000000
13      MIQ                0.000000
12      MPS                0.000000
11      CHI                0.000000

[708 rows x 2 columns]


In [191]:
betweenness_centrality_nx = nx.betweenness_centrality(G, weight='weight', normalized=True)

# Confronto
for node in G.nodes:
    print(f"Node: {node}")
    print(f"Manual: {betweenness_centrality_weighted[node]:.5f}")
    print(f"NetworkX: {betweenness_centrality_nx[node]:.5f}")
    print()

Node: MHK
Manual: 0.00812
NetworkX: 0.00669

Node: AMW
Manual: 0.00000
NetworkX: 0.00000

Node: EUG
Manual: 0.00360
NetworkX: 0.00324

Node: RDM
Manual: 0.00439
NetworkX: 0.00398

Node: MFR
Manual: 0.00511
NetworkX: 0.00393

Node: SEA
Manual: 0.02452
NetworkX: 0.01325

Node: PDX
Manual: 0.05855
NetworkX: 0.03088

Node: LMT
Manual: 0.00178
NetworkX: 0.00122

Node: SFO
Manual: 0.00616
NetworkX: 0.00346

Node: LAX
Manual: 0.01874
NetworkX: 0.01002

Node: EAT
Manual: 0.00815
NetworkX: 0.00573

Node: YKM
Manual: 0.00555
NetworkX: 0.00410

Node: EKO
Manual: 0.00896
NetworkX: 0.00822

Node: SLE
Manual: 0.00230
NetworkX: 0.00175

Node: GEG
Manual: 0.02211
NetworkX: 0.01683

Node: RDD
Manual: 0.00375
NetworkX: 0.00314

Node: LWS
Manual: 0.00869
NetworkX: 0.00594

Node: AST
Manual: 0.00000
NetworkX: 0.00000

Node: CLM
Manual: 0.00129
NetworkX: 0.00129

Node: PDT
Manual: 0.00634
NetworkX: 0.00373

Node: SJC
Manual: 0.00484
NetworkX: 0.00321

Node: ACV
Manual: 0.00108
NetworkX: 0.00090

Node: PUW


In [ ]:
# Funzione per calcolare la Closeness Centrality ponderata
def calculate_closeness_centrality(graph):
    """
    Calcola la Closeness Centrality ponderata per un grafo orientato.

    Parameters:
        graph (networkx.DiGraph): Il grafo rappresentante la rete di voli.

    Returns:
        dict: Dizionario contenente la Closeness Centrality per ogni nodo.
    """
    closeness = {}
    n = len(graph.nodes)  # Numero totale di nodi nel grafo

    for node in graph.nodes:
        # Inizializza le distanze
        dist = defaultdict(lambda: float('inf'))
        dist[node] = 0
        queue = []
        heapq.heappush(queue, (0, node))  # (distanza, nodo)

        # Dijkstra per calcolare le distanze minime
        while queue:
            d, current = heapq.heappop(queue)
            if d > dist[current]:
                continue
            for neighbor in graph.successors(current):
                weight = graph[current][neighbor]['weight']
                if dist[neighbor] > dist[current] + weight:
                    dist[neighbor] = dist[current] + weight
                    heapq.heappush(queue, (dist[neighbor], neighbor))

        # Calcola la somma delle distanze verso tutti gli altri nodi
        reachable_nodes = [d for d in dist.values() if d < float('inf')]
        sum_distances = sum(reachable_nodes)

        # Calcola la Closeness Centrality (evita la divisione per 0)
        if sum_distances > 0 and len(reachable_nodes) > 1:
            closeness[node] = (len(reachable_nodes) - 1) / sum_distances
        else:
            closeness[node] = 0.0  # Nessun nodo raggiungibile o nodo isolato

    return closeness
#
#
## Calcolo della Closeness Centrality
#closeness_centrality = calculate_closeness_centrality(G)
#
## Visualizza i risultati
#closeness_df = pd.DataFrame(closeness_centrality.items(), columns=['Airport', 'Closeness_Centrality'])
#print(closeness_df.sort_values(by='Closeness_Centrality', ascending=False, inplace=False))

    Airport  Closeness_Centrality
484     FVS              0.007463
128     STL              0.001097
134     CMI              0.001096
118     DEC              0.001094
115     SPI              0.001093
..      ...                   ...
668     SME              0.000000
672     AYS              0.000000
669     STK              0.000000
676     ARB              0.000000
726     PHD              0.000000

[727 rows x 2 columns]


In [ ]:
# Calcolo della Closeness Centrality con NetworkX
closeness_centrality_nx = nx.closeness_centrality(G, distance='weight')

# Conversione in DataFrame per NetworkX
closeness_centrality_nx_df = pd.DataFrame(
    closeness_centrality_nx.items(),
    columns=['Airport', 'Closeness_Centrality_NX']
)

# Confronto con i risultati manuali
comparison_df = pd.merge(closeness_df, closeness_centrality_nx_df, on='Airport')

# Visualizzazione del confronto
print(comparison_df.sort_values(by='Airport', ascending=True))

# Salvataggio per analisi
comparison_df.to_csv("closeness_centrality_comparison.csv", index=False)


In [ ]:
#Page rank

def calculate_pagerank(graph, degree_df, alpha=0.85, max_iter=100, tol=1.0e-6):
    nodes = list(graph.nodes)
    n = len(nodes)
    pagerank = {node: 1 / n for node in nodes}  # Inizializza PR(v) = 1 / N
    damping_value = (1 - alpha) / n  # Valore costante del damping

    # Creazione di un dizionario per il lookup rapido del grado uscente
    out_degree_dict = dict(zip(degree_df['Origin_airport'], degree_df['Out_degree']))

    for iteration in range(max_iter):
        new_pagerank = {}
        for node in nodes:
            # Somma dei contributi dai nodi predecessori
            rank_sum = sum(
                pagerank[neighbor] / out_degree_dict.get(neighbor, 1)  # Usa 1 se il nodo non ha out-degree
                for neighbor in graph.predecessors(node)
            )
            # Calcolo del nuovo valore di PageRank
            new_pagerank[node] = damping_value + alpha * rank_sum

        # Verifica della convergenza
        diff = sum(abs(new_pagerank[node] - pagerank[node]) for node in nodes)
        if diff < tol:
            print(f"Convergenza raggiunta in {iteration + 1} iterazioni.")
            break

        pagerank = new_pagerank

    return pagerank

# Calcolo del PageRank utilizzando l'Out_degree pre-calcolato
pagerank = calculate_pagerank(G, degree_df)

# Creazione di un DataFrame per i risultati
pagerank_df = pd.DataFrame(pagerank.items(), columns=['Airport', 'PageRank'])

# Visualizza i risultati
print(pagerank_df.sort_values(by='PageRank', ascending=False))


In [242]:
def calculate_pagerank(graph, alpha=0.85, max_iter=100, tol=1.0e-6):
    """
    Calcola il PageRank utilizzando il grado uscente calcolato con calculate_degree_centrality.

    Parameters:
        graph (networkx.DiGraph): Il grafo rappresentante la rete di voli.
        alpha (float): Fattore di damping (default: 0.85).
        max_iter (int): Numero massimo di iterazioni (default: 100).
        tol (float): Tolleranza per la convergenza (default: 1.0e-6).

    Returns:
        dict: Dizionario contenente il PageRank di ogni nodo.
    """
    nodes = list(graph.nodes)
    n = len(nodes)
    pagerank = {node: 1 / n for node in nodes}  # Inizializza PR(v) = 1 / N
    damping_value = (1 - alpha) / n

    # Calcolo del grado uscente usando calculate_degree_centrality
    degree_df = calculate_degree_centrality(graph)
    out_degree = degree_df.set_index('Airport')['Out_degree'].to_dict()

    for iteration in range(max_iter):
        new_pagerank = {}
        for node in nodes:
            # Somma dei contributi dai nodi predecessori
            rank_sum = sum(
                pagerank[neighbor] / out_degree.get(neighbor, 1)  # Usa il grado uscente calcolato
                for neighbor in graph.predecessors(node)
            )
            # Calcolo del nuovo valore di PageRank
            new_pagerank[node] = damping_value + alpha * rank_sum

        # Verifica della convergenza
        diff = sum(abs(new_pagerank[node] - pagerank[node]) for node in nodes)
        if diff < tol:
            break

        pagerank = new_pagerank

    return pagerank
#
pagerank_df = pd.DataFrame(calculate_pagerank(G).items(), columns=['Airport', 'PageRank'])
#
## Visualizza i risultati
print(pagerank_df.sort_values(by='PageRank', ascending=False))

    Airport  PageRank
196     YIP  0.011969
99      SHV  0.007199
104     MSP  0.007116
131     MEM  0.006640
100     MCI  0.006623
..      ...       ...
667     PMH  0.000206
719     LXN  0.000206
716     MHE  0.000206
714     DQC  0.000206
680     HCA  0.000206

[727 rows x 2 columns]


In [243]:
def analyze_centrality(flight_network, airport):
    """
    Compute only the Betweenness Centrality for a given airport.

    Parameters:
        flight_network (networkx.DiGraph): The flight network as a directed graph.
        airport (str): The airport for which centrality measures are computed.

    Returns:
        dict: A dictionary with Betweenness Centrality.
    """
    # Calcolo della Betweenness Centrality ponderata
    betweenness_centrality_weighted = calculate_betweenness_centrality_weighted(flight_network)
    # Calcolo della Closeness Centrality ponderata
    closeness_centrality = calculate_closeness_centrality(flight_network)
    # Calcolo del Degree Centrality
    degree_df = calculate_degree_centrality(flight_network)
    degree_centrality = degree_df.set_index('Airport').to_dict(orient='index')
    # Calcolo del PageRank
    pagerank = calculate_pagerank(flight_network)

    return {
        'Betweenness Centrality': betweenness_centrality_weighted.get(airport, 0.0),
        'Closeness Centrality': closeness_centrality.get(airport, 0.0), 
        'Degree Centrality': degree_centrality.get(airport, {}).get('Total_degree', 0),
        'PageRank': pagerank.get(airport, 0.0)
    }



In [244]:
analyze_centrality(G, "YIP")

{'Betweenness Centrality': 0.32333452216274433,
 'Closeness Centrality': 0.00108300373460128,
 'Degree Centrality': 665,
 'PageRank': 0.011968947291634198}

Ask LLM (eg. ChatGPT) to suggest alternative centrality measures that might be relevant to this task. How can you check that the results given by the LLM are trustable?


La Katz Centrality è una misura di centralità usata per identificare l'importanza di un nodo in una rete, prendendo in considerazione sia il numero di connessioni dirette (ad esempio, i vicini immediati) sia le connessioni indirette (i vicini dei vicini, e così via).

Caratteristiche principali
- Un nodo è importante non solo se ha molte connessioni, ma anche se è connesso a nodi importanti (simile alla Eigenvector Centrality).
- Include un bias: La Katz Centrality aggiunge un termine di bias (detto beta) per garantire che anche nodi con poche connessioni abbiano un punteggio positivo. Questo è utile per grafi disconnessi o reti sparse.


La **Katz Centrality** di un nodo \( v \) è definita come:

$$
C_k(v) = \alpha \sum_{u \in N(v)} C_k(u) + \beta
$$

Dove:
- \( C_k(v) \): La Katz Centrality del nodo \( v \).
- \( N(v) \): I vicini del nodo \( v \) (i nodi direttamente collegati a \( v \)).
- \( alpha \): Un fattore di attenuazione che controlla l'influenza delle connessioni indirette.
- \( beta \): Un termine di bias per garantire che i nodi senza connessioni abbiano un punteggio positivo.

In [250]:
nx.katz_centrality(G, alpha=0.01, beta=1.0, max_iter=1000, tol=1e-6, weight='weight')

PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 1000 iterations')